# Republican, Democrat, or Sesame Tweet Predictor

This notebook outputs a Stocastic Gradient Decent model for predicting the pollitical party of a tweeter.  Specifically, it outputs the following files:


In [1]:
# Importing libraries
import pandas as pd

# Tools to remove stopwords from tweets
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package stopwords to C:\Users\Herman D
[nltk_data]     Schaumburg\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Herman D
[nltk_data]     Schaumburg\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
# Setting up stop words and defining functions.
stop_words = set(stopwords.words('english'))

def fix_party_code(pc):
    if pc == 100:
        return 'D'
    elif pc == 200:
        return 'R'
    else:
        return 'S'
#    return int(pc/100-1)
def list_tostring(input_list):
    return ' '.join(input_list)

def remove_stopwords(input_list):
    return [w for w in input_list if not w in stop_words]

def clean_tweets(input_df):
    input_df['party_code'] = input_df['party_code'].apply(fix_party_code)
    input_df['text'] = input_df['text'].apply(word_tokenize)
    input_df['text'] = input_df['text'].apply(remove_stopwords)
    input_df['text'] = input_df['text'].apply(list_tostring)    
    return input_df

## Adding R and D tweets to training dataframe

In [3]:
# Read csv with party affiliations
party_code_df = pd.read_csv("partycode.csv")
party_code_df.head()

,account_id,bioguide_id,mem_name,screen_name,chamber,state_abbr,party_code
0,37007274,Y000033,Don Young,repdonyoung,House,AK,200
1,2559398984,Y000033,Don Young,DonYoungAK,House,AK,200
2,2253968388,B001289,Bradley Byrne,RepByrne,House,AL,200
3,42481696,B001289,Bradley Byrne,BradleyByrne,House,AL,200
4,2861616083,P000609,Gary Palmer,USRepGaryPalmer,House,AL,200


In [4]:
party_dict = party_code_df[['party_code','account_id']].set_index('account_id').to_dict()
party_dict = party_dict['party_code']

In [5]:
# Specify dates for training data
dates = pd.date_range(start='7/01/2020', end='7/10/2020')
dates = [str(date)[0:10] for date in dates]

# Specify number of samples per day per party
# date_samples = 1000
date_samples = 100
print('Training data consists of '+str(date_samples*1*len(dates))+' tweets per party(R,D,S).')

Training data consists of 1000 tweets per party(R,D,S).


In [6]:
date = dates[0]
date

'2020-07-01'

In [7]:
# Create empty dataframe
train_df = pd.DataFrame(columns=['text','party_code'])

for date in dates:
    date_str = date
    print(date)
    example_tweets = pd.read_json("congresstweets/data/"+date_str+".json")
    example_tweets["party_code"] = example_tweets["user_id"]
    example_tweets = example_tweets.replace({"party_code":party_dict})
    date_train_data = example_tweets[(example_tweets['party_code']!=100)|(example_tweets['party_code']!=200)][['text','party_code']]
    # Rearrange rows
    date_train_data = date_train_data.sample(frac=1)
    date_train_data_R=date_train_data[date_train_data['party_code']==200].head(date_samples)
    date_train_data_D=date_train_data[date_train_data['party_code']==100].head(date_samples)
    # Remove stop words and fix party code
    date_train_data = clean_tweets(date_train_data_D.append(date_train_data_R, ignore_index=True))
    #if date == date[0]:
    #    train_df = date_train_data
    #else:
    train_df = train_df.append(date_train_data)#, ignore_index=True)

2020-07-01
2020-07-02
2020-07-03
2020-07-04
2020-07-05
2020-07-06
2020-07-07
2020-07-08
2020-07-09
2020-07-10


In [8]:
train_df = train_df.reset_index(drop=True)

In [9]:
sesame_tweets = pd.read_csv("sesametweets/data/sesame_tweets.csv", usecols = ['text'])
sesame_tweets = sesame_tweets.assign(party_code='S')
sesame_tweets = clean_tweets(sesame_tweets)

train_sesame_tweets = sesame_tweets.head(1000)

train_df = train_df.append(train_sesame_tweets)
train_df = train_df.sample(frac=1)
train_df = train_df.reset_index(drop=True)

# Text Vectorization

In [10]:
from os import system
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from scipy.sparse import csr_matrix
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from joblib import dump, load # used for saving and loading sklearn objects
from scipy.sparse import save_npz, load_npz # used for saving and loading sparse matrices

In [11]:
system("mkdir data_preprocessors")
system("mkdir vectorized_data")

# Unigram Counts
unigram_vectorizer = CountVectorizer(ngram_range=(1, 1))
unigram_vectorizer.fit(train_df['text'].values)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [12]:
dump(unigram_vectorizer, 'data_preprocessors/unigram_vectorizer.joblib')

['data_preprocessors/unigram_vectorizer.joblib']

In [13]:
# unigram_vectorizer = load('data_preprocessors/unigram_vectorizer.joblib')

X_train_unigram = unigram_vectorizer.transform(train_df['text'].values)
save_npz('vectorized_data/X_train_unigram.npz', X_train_unigram)

In [14]:
# Unigram Tf-Idf

unigram_tf_idf_transformer = TfidfTransformer()
unigram_tf_idf_transformer.fit(X_train_unigram)

dump(unigram_tf_idf_transformer, 'data_preprocessors/unigram_tf_idf_transformer.joblib')

['data_preprocessors/unigram_tf_idf_transformer.joblib']

In [15]:
X_train_unigram_tf_idf = unigram_tf_idf_transformer.transform(X_train_unigram)

save_npz('vectorized_data/X_train_unigram_tf_idf.npz', X_train_unigram_tf_idf)

In [16]:
# Bigram Counts

bigram_vectorizer = CountVectorizer(ngram_range=(1, 3))
bigram_vectorizer.fit(train_df['text'].values)

dump(bigram_vectorizer, 'data_preprocessors/bigram_vectorizer.joblib')

['data_preprocessors/bigram_vectorizer.joblib']

In [17]:
X_train_bigram = bigram_vectorizer.transform(train_df['text'].values)

save_npz('vectorized_data/X_train_bigram.npz', X_train_bigram)

In [18]:
bigram_tf_idf_transformer = TfidfTransformer()
bigram_tf_idf_transformer.fit(X_train_bigram)

dump(bigram_tf_idf_transformer, 'data_preprocessors/bigram_tf_idf_transformer.joblib')

['data_preprocessors/bigram_tf_idf_transformer.joblib']

In [19]:
X_train_bigram_tf_idf = bigram_tf_idf_transformer.transform(X_train_bigram)

save_npz('vectorized_data/X_train_bigram_tf_idf.npz', X_train_bigram_tf_idf)

# SGDClassifier

In [20]:
def train_and_show_scores(X: csr_matrix, y: np.array, title: str) -> None:
    X_train, X_valid, y_train, y_valid = train_test_split(
        X, y, train_size=0.75, stratify=y
    )

    clf = SGDClassifier()
    clf.fit(X_train, y_train)
    train_score = clf.score(X_train, y_train)
    valid_score = clf.score(X_valid, y_valid)
    print(f'{title}\nTrain score: {round(train_score, 3)} ; Validation score: {round(valid_score, 3)}\n')

y_train = train_df['party_code'].values

In [21]:
train_and_show_scores(X_train_unigram, y_train, 'Unigram Counts')
train_and_show_scores(X_train_unigram_tf_idf, y_train, 'Unigram Tf-Idf')
train_and_show_scores(X_train_bigram, y_train, 'Bigram Counts')
train_and_show_scores(X_train_bigram_tf_idf, y_train, 'Bigram Tf-Idf')

Unigram Counts
Train score: 1.0 ; Validation score: 0.795

Unigram Tf-Idf
Train score: 0.999 ; Validation score: 0.805

Bigram Counts
Train score: 1.0 ; Validation score: 0.789

Bigram Tf-Idf
Train score: 1.0 ; Validation score: 0.837



# Using Cross-Validation for hyperparameter tuning

In [22]:

from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

X_train = X_train_bigram_tf_idf


# Phase 1: loss, learning rate and initial learning rate

clf = SGDClassifier()

distributions = dict(
    loss=['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron'],
    learning_rate=['optimal', 'invscaling', 'adaptive'],
    eta0=uniform(loc=1e-7, scale=1e-2)
)

random_search_cv = RandomizedSearchCV(
    estimator=clf,
    param_distributions=distributions,
    cv=5,
    n_iter=100
)
random_search_cv.fit(X_train, y_train)
print(f'Best params: {random_search_cv.best_params_}')
print(f'Best score: {random_search_cv.best_score_}')

Best params: {'eta0': 0.007666261216410279, 'learning_rate': 'optimal', 'loss': 'hinge'}
Best score: 0.8360000000000001


In [23]:
# Phase 2: penalty and alpha

clf = SGDClassifier()

distributions = dict(
    penalty=['l1', 'l2', 'elasticnet'],
    alpha=uniform(loc=1e-6, scale=1e-4)
)

random_search_cv = RandomizedSearchCV(
    estimator=clf,
    param_distributions=distributions,
    cv=5,
    n_iter=50
)
random_search_cv.fit(X_train, y_train)
print(f'Best params: {random_search_cv.best_params_}')
print(f'Best score: {random_search_cv.best_score_}')

Best params: {'alpha': 8.412554870901323e-05, 'penalty': 'l2'}
Best score: 0.8373333333333333


## Saving best classifier

In [24]:
system("mkdir classifiers")

sgd_classifier = random_search_cv.best_estimator_

dump(random_search_cv.best_estimator_, 'classifiers/sgd_classifier.joblib')

# sgd_classifier = load('classifiers/sgd_classifier.joblib')

['classifiers/sgd_classifier.joblib']

# Testing Classifier

In [25]:
sesame_tweets.shape

(1319, 2)

In [26]:
# Selecte Sesame Tweets
num_test_tweets = 319
test_sesame_tweets = sesame_tweets.tail(319)

In [27]:
# Gather test data
# Create empty dataframe
test_df = pd.DataFrame(columns=['text','party_code'])

# Specify dates for test data
test_dates = pd.date_range(start='8/2/2020', end='8/3/2020')
test_dates = [str(date)[0:10] for date in dates]

for date in test_dates:
    date_str = date
    example_tweets = pd.read_json("congresstweets/data/"+date_str+".json")
    example_tweets["party_code"] = example_tweets["user_id"]
    example_tweets = example_tweets.replace({"party_code":party_dict})
    date_test_data = example_tweets[(example_tweets['party_code']!=100)|(example_tweets['party_code']!=200)][['text','party_code']]
    date_test_data = clean_tweets(date_test_data)
    #if date == date[0]:
    #    train_df = date_train_data
    #else:
    test_df = test_df.append(date_test_data)#, ignore_index=True)
test_df = test_df.sample(frac=1).reset_index(drop=True).head(2*num_test_tweets)

In [28]:
test_df = test_df.append(test_sesame_tweets)
test_df = test_df.reset_index(drop=True)
test_df = test_df.sample(frac=1)

In [29]:
X_test = bigram_vectorizer.transform(test_df['text'].values)
X_test = bigram_tf_idf_transformer.transform(X_test)
#X_test = unigram_tf_idf_transformer.transform(X_test)
y_test = test_df['party_code'].values

score = sgd_classifier.score(X_test, y_test)
print(score)

0.7189132706374086


In [30]:
# Mitch McConnell
in_text = "This new bill strengthens some of the most important parts of the CARES Act, especially the popular Paycheck Protection Program which is saving millions of American jobs as we speak. And Republicans successfully kept extraneous issues out of the bill. It’s a win for the country."
fun_input = list_tostring(word_tokenize(in_text))
unigram_vectorizer = load('data_preprocessors/unigram_vectorizer.joblib')
X_pred = bigram_vectorizer.transform([fun_input])
X_pred = bigram_tf_idf_transformer.transform(X_pred)
sgd_classifier = load('classifiers/sgd_classifier.joblib')

result = sgd_classifier.predict(X_pred)
result[0]

'R'

In [32]:
# Chuck Schumer
in_text = "Amid coronavirus—Trump admin's actively working to roll back critical protections against predatory lenders We’ll fight this—we need to crack down on loan sharks preying on the most vulnerable consumers Not let them peddle interest rates as high as 800%!"
fun_input = list_tostring(word_tokenize(in_text))
unigram_vectorizer = load('data_preprocessors/unigram_vectorizer.joblib')
X_pred = bigram_vectorizer.transform([fun_input])
X_pred = bigram_tf_idf_transformer.transform(X_pred)
sgd_classifier = load('classifiers/sgd_classifier.joblib')

result = sgd_classifier.predict(X_pred)
result[0]

'D'

In [33]:
# Cookie Monster
in_text = "Ice cream truck passed by today. Why there no cookie trucks? Could be new business opportunity!"
fun_input = list_tostring(word_tokenize(in_text))
unigram_vectorizer = load('data_preprocessors/unigram_vectorizer.joblib')
X_pred = bigram_vectorizer.transform([fun_input])
X_pred = bigram_tf_idf_transformer.transform(X_pred)
sgd_classifier = load('classifiers/sgd_classifier.joblib')

result = sgd_classifier.predict(X_pred)
result[0]

'S'